In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
# Helper functions
def random_date(start, end):
    return start + timedelta(seconds=np.random.randint(0, int((end - start).total_seconds())))

def generate_sample_dates(start_date, end_date, num_records):
    return [random_date(start_date, end_date).strftime('%Y-%m-%d') for _ in range(num_records)]

def generate_time():
    time = datetime.strptime('00:00', '%H:%M') + timedelta(minutes=np.random.randint(0, 1440))
    return time.strftime('%H:%M')

In [6]:
# Use context manager to write Excel file
with pd.ExcelWriter('../data/sample_data.xlsx', engine='openpyxl') as writer:

    # 1. Daily DFR
    pd.DataFrame({
        'REPORT_DATE': generate_sample_dates(datetime(2021, 4, 1), datetime(2025, 4, 30), 20),
        'START_TIME': [generate_time() for _ in range(20)],
        'FINISH_TIME': [generate_time() for _ in range(20)],
        'ELAPSED_HOURS': np.random.uniform(1, 12, 20).round(2)
    }).to_excel(writer, sheet_name='Daily DFR', index=False)

    # 2. Volume All
    pd.DataFrame({
        'Date': generate_sample_dates(datetime(2021, 4, 1), datetime(2025, 4, 30), 20),
        'Auth': np.random.randint(1000, 5000, 20),
        'Txn': np.random.randint(100, 1000, 20)
    }).to_excel(writer, sheet_name='Volume All', index=False)

    # 3. Volume (Subm)-Top Merchants
    pd.DataFrame({
        'Month': pd.date_range('2021-04-01', periods=20, freq='M').strftime('%Y-%m'),
        'Metropolitan (MTA)': np.random.randint(10000, 50000, 20)
    }).to_excel(writer, sheet_name='Volume (Subm)-Top Merchants', index=False)

    # For 3-year monthly data sheets (April 2021-April 2025)
    months = pd.date_range('2021-04-01', '2025-04-30', freq='M').strftime('%Y-%m').tolist()
    entities = ['Sales', 'Marketing', 'Finance']
    companies = ['Company A', 'Company B', 'Company C']
    reports = ['Monthly Sales', 'Quarterly Financial', 'Marketing Analysis']
    frequencies = ['Monthly', 'Weekly', 'Daily']

    common_data = {
        'COMPANY_ID': np.random.randint(1000, 9999, 20),
        'ENTITY': np.random.choice(entities, 20),
        'BREAKDOWN_LVL': np.random.choice(['High', 'Medium', 'Low'], 20),
        'ENTITY_ID': np.random.randint(100, 999, 20),
        'DB_KEY': np.random.randint(100000, 999999, 20),
        'COMPANY_NAME': np.random.choice(companies, 20),
        'REPORT_NAME': np.random.choice(reports, 20),
        'REPORT_DATE': np.random.choice(months, 20),
        'FREQUENCY': np.random.choice(frequencies, 20),
        'START_TIME': [generate_time() for _ in range(20)],
        'FINISH_TIME': [generate_time() for _ in range(20)],
        'ELAPSED_HOURS': np.random.uniform(1, 24, 20).round(2)
    }

    # Generate sheets
    sheet_names = [
        'Daily Top Web Reports', 'Daily Top GRPT DFR Reports', 
        'Top monthly web reports', 'Top monthly Grpt DFR reports (>10 hrs)'
    ]

    for name in sheet_names:
        data = common_data.copy()
        if 'BREAKDOWN_LVL' not in name:
            data.pop('BREAKDOWN_LVL')
        df = pd.DataFrame(data)
        if name == 'Top monthly Grpt DFR reports (>10 hrs)':
            df = df[df['ELAPSED_HOURS'] > 10]
        df.to_excel(writer, sheet_name=name, index=False)

    # Monthend Duration-Web, Grpt
    pd.DataFrame({
        'REPORT_DATE': months[:20],
        'START_TIME': [generate_time() for _ in range(20)],
        'FINISH_TIME': [generate_time() for _ in range(20)],
        'MAX_DURATION_HOURS': np.random.uniform(5, 20, 20).round(2)
    }).to_excel(writer, sheet_name='Monthend Duration Web GRPT', index=False)

    # Daily Web
    pd.DataFrame({
        'REPORT_DATE': generate_sample_dates(datetime(2021, 4, 1), datetime(2025, 4, 30), 20),
        'START_TIME': [generate_time() for _ in range(20)],
        'FINISH_TIME': [generate_time() for _ in range(20)],
        'ELAPSED_HOURS': np.random.uniform(1, 12, 20).round(2)
    }).to_excel(writer, sheet_name='Daily Web', index=False)

print("Excel file 'sample_data.xlsx' created successfully!")

Excel file 'sample_data.xlsx' created successfully!


/var/folders/bn/r3jcwdgd6ss8l34kf4p54z9h0000gn/T/ipykernel_24863/3613060447.py:21: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  'Month': pd.date_range('2021-04-01', periods=20, freq='M').strftime('%Y-%m'),
/var/folders/bn/r3jcwdgd6ss8l34kf4p54z9h0000gn/T/ipykernel_24863/3613060447.py:26: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  months = pd.date_range('2021-04-01', '2025-04-30', freq='M').strftime('%Y-%m').tolist()
/Users/sumandey/anaconda3/envs/graph_rag/lib/python3.11/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [1]:
from langchain_community.document_loaders import UnstructuredExcelLoader

In [29]:
loader = UnstructuredExcelLoader("../data/sample_data.xlsx", mode="elements")
docs = loader.load()

print(len(docs))

9


In [30]:
docs[0].metadata


{'source': '../data/sample_data.xlsx',
 'filename': 'sample_data.xlsx',
 'file_directory': '../data',
 'last_modified': '2025-05-09T09:56:10',
 'filetype': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
 'languages': ['eng'],
 'page_number': 1,
 'page_name': 'Daily DFR',
 'text_as_html': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>REPORT_DATE</td>\n      <td>START_TIME</td>\n      <td>FINISH_TIME</td>\n      <td>ELAPSED_HOURS</td>\n    </tr>\n    <tr>\n      <td>2023-05-18</td>\n      <td>07:07</td>\n      <td>12:50</td>\n      <td>11.49</td>\n    </tr>\n    <tr>\n      <td>2025-01-26</td>\n      <td>02:31</td>\n      <td>16:07</td>\n      <td>10.84</td>\n    </tr>\n    <tr>\n      <td>2023-08-13</td>\n      <td>21:46</td>\n      <td>13:45</td>\n      <td>6.66</td>\n    </tr>\n    <tr>\n      <td>2022-03-08</td>\n      <td>13:58</td>\n      <td>12:40</td>\n      <td>4.54</td>\n    </tr>\n    <tr>\n      <td>2024-02-08</td>\n      <td>06:4

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=80)
chunks = text_splitter.split_documents(docs)

In [25]:
chunks

[Document(page_content='REPORT_DATE\nSTART_TIME\nFINISH_TIME\nELAPSED_HOURS\n\n\n2023-05-18\n07:07\n12:50\n11.49\n\n\n2025-01-26\n02:31\n16:07\n10.84\n\n\n2023-08-13\n21:46\n13:45\n6.66\n\n\n2022-03-08\n13:58\n12:40\n4.54\n\n\n2024-02-08\n06:44\n21:30\n9.64\n\n\n2025-04-01\n02:22\n06:46\n6.68\n\n\n2024-07-22\n09:08\n19:09\n6.75\n\n\n2024-09-14\n16:38\n04:10\n7.4\n\n\n2023-02-07\n20:17\n03:42\n1.88\n\n\n2024-01-08\n09:10\n00:21\n4.27\n\n\n2021-09-27\n09:30\n12:47\n5.59\n\n\n2023-12-12\n20:37\n02:11\n6.5\n\n\n2023-12-19\n19:47\n22:41\n11.02\n\n\n2022-01-15\n20:48\n02:32\n9.07\n\n\n2024-02-10\n14:28\n10:52\n7.96\n\n\n2021-11-08\n11:28\n03:58\n7.68\n\n\n2024-11-15\n12:44\n15:35\n2.1\n\n\n2025-01-28\n09:38\n16:59\n4.83\n\n\n2023-07-14\n09:24\n16:55\n7.69\n\n\n2024-12-24\n23:47\n22:21\n6.72\n\n\n\n\n\n\n\nDate\nAuth\nTxn\n\n\n2023-06-08\n1284\n458\n\n\n2023-05-21\n3044\n834\n\n\n2024-01-04\n3929\n787\n\n\n2021-09-21\n4647\n375\n\n\n2025-02-21\n4711\n791\n\n\n2023-01-28\n3526\n551\n\n\n2021-1

In [37]:
chunks[0]

Document(page_content='REPORT_DATE\nSTART_TIME\nFINISH_TIME\nELAPSED_HOURS\n\n\n2023-05-18\n07:07\n12:50\n11.49\n\n\n2025-01-26\n02:31\n16:07\n10.84\n\n\n2023-08-13\n21:46\n13:45\n6.66\n\n\n2022-03-08\n13:58\n12:40\n4.54\n\n\n2024-02-08\n06:44\n21:30\n9.64\n\n\n2025-04-01\n02:22\n06:46\n6.68\n\n\n2024-07-22\n09:08\n19:09\n6.75\n\n\n2024-09-14\n16:38\n04:10\n7.4\n\n\n2023-02-07\n20:17\n03:42\n1.88\n\n\n2024-01-08\n09:10\n00:21\n4.27\n\n\n2021-09-27\n09:30\n12:47\n5.59\n\n\n2023-12-12\n20:37\n02:11\n6.5\n\n\n2023-12-19\n19:47\n22:41\n11.02\n\n\n2022-01-15\n20:48\n02:32\n9.07\n\n\n2024-02-10\n14:28\n10:52\n7.96\n\n\n2021-11-08\n11:28\n03:58\n7.68\n\n\n2024-11-15\n12:44\n15:35\n2.1\n\n\n2025-01-28\n09:38\n16:59\n4.83\n\n\n2023-07-14\n09:24\n16:55\n7.69\n\n\n2024-12-24\n23:47\n22:21\n6.72\n\n\n\n\n\n\n\nDate\nAuth\nTxn\n\n\n2023-06-08\n1284\n458\n\n\n2023-05-21\n3044\n834\n\n\n2024-01-04\n3929\n787\n\n\n2021-09-21\n4647\n375\n\n\n2025-02-21\n4711\n791\n\n\n2023-01-28\n3526\n551\n\n\n2021-11

In [33]:
print(docs[0])

page_content='\n\n\nREPORT_DATE\nSTART_TIME\nFINISH_TIME\nELAPSED_HOURS\n\n\n2023-05-18\n07:07\n12:50\n11.49\n\n\n2025-01-26\n02:31\n16:07\n10.84\n\n\n2023-08-13\n21:46\n13:45\n6.66\n\n\n2022-03-08\n13:58\n12:40\n4.54\n\n\n2024-02-08\n06:44\n21:30\n9.64\n\n\n2025-04-01\n02:22\n06:46\n6.68\n\n\n2024-07-22\n09:08\n19:09\n6.75\n\n\n2024-09-14\n16:38\n04:10\n7.4\n\n\n2023-02-07\n20:17\n03:42\n1.88\n\n\n2024-01-08\n09:10\n00:21\n4.27\n\n\n2021-09-27\n09:30\n12:47\n5.59\n\n\n2023-12-12\n20:37\n02:11\n6.5\n\n\n2023-12-19\n19:47\n22:41\n11.02\n\n\n2022-01-15\n20:48\n02:32\n9.07\n\n\n2024-02-10\n14:28\n10:52\n7.96\n\n\n2021-11-08\n11:28\n03:58\n7.68\n\n\n2024-11-15\n12:44\n15:35\n2.1\n\n\n2025-01-28\n09:38\n16:59\n4.83\n\n\n2023-07-14\n09:24\n16:55\n7.69\n\n\n2024-12-24\n23:47\n22:21\n6.72\n\n\n' metadata={'source': '../data/sample_data.xlsx', 'filename': 'sample_data.xlsx', 'file_directory': '../data', 'last_modified': '2025-05-09T09:56:10', 'filetype': 'application/vnd.openxmlformats-officedo